<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/ai4eo_logos.png' align='right' width='100%'/>

<a href="../../../Index.ipynb"><< Index</a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2024 European Union <br>
**License:** MIT <br>
**Authors:** Ben Loveday (EUMETSAT/Innoflair UG), Hayley Evers-King (EUMETSAT), Lauren Biermann (Plymouth University) <br>

<div class="alert alert-block alert-success">
<h3>Ocean colour applications</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has no prerequisites, but if you wish to download your own OLCI data you will need:<br>
- **<a href="https://eoportal.eumetsat.int/" target="_blank">A EUMETSAT Earth Observation Portal account</a>** if you are using or plan to use the EUMETSAT Data Store

</div>
<hr>

# Optical water type classification: a Naive Bayesian approach
<font color="#138D75">*This notebook is adapted from worklows developed under the 2022 <a href="https://www.futurelearn.com/courses/artificial-intelligence-for-earth-monitoring/" target="_blank">AI4EO MOOC</a>*</font>.

### Data used

| Dataset | EUMETSAT collection ID| EUMETSAT collection<br>description | WEkEO dataset ID | WEkEO description |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:-----------------:|
| Sentinel-3 OLCI level 2 full resolution | EO:EUM:DAT:0407 | <a href="https://user.eumetsat.int/catalogue/EO:EUM:DAT:SENTINEL-3:OL_2_WFR___NTC" target="_blank">Description</a> | EO:EUM:DAT:SENTINEL-3:OL_2_WFR___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3AOL_2_WFR___" target="_blank">Description</a> |
| Sentinel-3 OLCI level 1b full resolution | EO:EUM:DAT:0409 | <a href="https://user.eumetsat.int/catalogue/EO:EUM:DAT:SENTINEL-3:OL_1_EFR___NTC" target="_blank">Description</a> | EO:EUM:DAT:SENTINEL-3:OL_1_EFR___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3AOL_1_EFR___" target="_blank">Description</a> |

### Learning outcomes

At the end of this notebook you will know how to ;

* acquire Sentinel-3 OLCI level 1b and level 2 data from the EUMETSAT Data Store
* apply a Naive Bayesian Classifier to determine where complex water occur

### Outline

Coastal oceans are where society meets the sea. Coastal ocean waters provide a large proportion of the worlds food through aquaculture and fisheries, and are the site of many socially and economically important activities including tourism and recreation, ports and shipping, and renewable energy generation. Management of these regions is therefore vitally important to sustainable development. 

Coastal ocean water quality influences suitability of areas for fishing, aquaculture and tourism, and can be negatively impacted by various industries operating in the marine domain. Data is needed to monitor the impact of different industries, conduct marine spatial planning, and for compliance with policy such as the European water and marine strategy framework directives. Satellite ocean colour data can provide information on coastal water quality at temporal and spatial scales suitable for these activities. However, translating the measurements the satellite makes (essentially of light of different colours) into properties of interest (water transparency, biological activity, and sediment concentrations) is challenging, particularly in coastal regions.

This workflow guides you through classification of <a href="https://user.eumetsat.int/data/satellites/sentinel-3/ocean-colour-service" target="_blank">Sentinel-3 OLCI</a> ocean colour data to identify complex waters using a <a href="https://scikit-learn.org/stable/modules/naive_bayes.html" target="_blank">Naive Bayes Classifier</a> from the SciKit-Learn toolbox. This is a relatively simple method based on Bayes Theorem, that works fast and requires relatively small amounts of training data. The method has a key assumption that the features it classifies are independent.

This workflow uses the following data:
* An OLCI full resolution level 1 (EFR) scene for the North Sea 16.04.2021
* An OLCI full resolution level 2 (WFR) scene for the North Sea 16.04.2021
* The spectral output of a radiative transfer model (Modelled_spectral_reflectance_data.nc) to simulate remote sensing reflectances from a range of in water properties, adapted from <a href="https://www.osapublishing.org/oe/fulltext.cfm?uri=oe-22-10-11536&id=284439" target="_blank">Evers-King et al. 2014</a>

The workflow in this notebook involves the process of training your own naive bayesian classifier, and application to a standard OLCI Level-2 product. No pre-training is required.

### Further resources

1. <a href="https://user.eumetsat.int/resources/user-guides/sentinel-3-olci-level-1-data-guide" target="_blank">Sentinel-3 OLCI level 1 data guide</a>
2. <a href="https://user.eumetsat.int/resources/user-guides/sentinel-3-ocean-colour-level-2-data-guide" target="_blank">Sentinel-3 ocean colour level 2 data guide</a>
3. <a href="http://misclab.umeoce.maine.edu/documents/ocean_17.2_mobley_lo.pdf" target="_blank">Mobley et al. 2003 - Optical Modelling of Ocean Waters: Is the Case 1 Case 2 classification still useful?</a>
4. <a href="https://towardsdatascience.com/naive-bayes-classifier-81d512f50a7c" target="_blank">The Naive Bayes Classifier</a>
5. <a href="https://www.nature.com/articles/s41598-020-62298-z" target="_blank">Application of the Naive Bayes Classifier to marine debris detection</a>
6. <a href="https://www.osapublishing.org/oe/fulltext.cfm?uri=oe-22-10-11536&id=284439" target="_blank">Sensitivity in reflectance attributed to phytoplankton cell size: forward and inverse modelling approaches</a>
7. <a href="http://omtab.obs-vlfr.fr/fichiers_PDF/Defoin_and_chami_JGR_07.pdf" target="_blank">Defoin-Platel and Chami, 2007</a>.
  
<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
1. [Introduction to ocean colour and Copernicus Sentinel-3 OLCI marine data](#section1)
1. [Setting up our experiment](#section2)
1. [Downloading our data](#section3)
1. [Loading and checking our data](#section4) 
1. [Interpreting OLCI spectra](#section5)
1. [Example: Classifying spectra with a Naive Bayesian Classifier](#section6)
1. [Conclusions](#section7)

<div class="alert alert-info" role="alert">

## 1. <a id='section1'></a>Introduction to ocean colour and Copernicus Sentinel-3 OLCI marine data
[Back to top](#TOC_TOP)

</div>

The ocean colour signal is made up of light reflected by the different components in the ocean waters.  We can get a rough idea of what is in ocean waters just by looking at the colour with our own eyes - we associate blue with water, brown with sediments, and green with plant-life (in the ocean, this green comes from plant-like organisms called phytoplankton). However, if we want to know how much sediment or phytoplankton are in a particular patch of ocean, this is not something we can easily estimate visually. In coastal waters, this is even more complicated because there is often a mix of different things in the water, and situations can change rapidly in space and time thanks to the dynamic nature of the coastal oceans. Coastal currents, algal blooms, increased river flow from flooding, storms, pollution, all make the colour of the coastal ocean highly variable.

The Ocean and Land Colour Instruments (OLCI) aboard the Sentinel-3 satellites provides near daily coverage, at 300 m resolution, using a high number of narrow spectral bands with suitable signal to noise ratio for the oceans. All of these characteristics help towards getting the most accurate data about the colour of the coastal ocean but we still have a challenge to turn the signal in to the information that decision makers need, and this is where machine learning can help.

Ocean colour data from OLCI is available from EUMETSAT at two levels - level-1 and level-2. Level-1 data is the top-of-atmosphere product where the signal has not yet been corrected for the influence of the atmosphere. At level-2 atmospheric correction has been performed so that you have the signal from the water, as well as some geophysical properties that have been derived from it. OLCI data products include the 'colour' data which is properly referred to as measurements of radiance or reflectance at the different wavelengths of light that the OLCI instrument measures at. OLCI measures at 21 wavelengths and has 21 spectral bands, although not all of these are used for parameter retrieval, some are solely used for atmospheric correction. 

You can find information about OLCI data formats and processing in the <a href="https://user.eumetsat.int/resources/user-guides/sentinel-3-ocean-colour-level-2-data-guide" target="_blank">Sentinel-3 ocean colour level 2 data guide</a>. You will also find OLCI data integrated in to other products such as those available from the Copernicus services. We are choosing to use the level 1b and level 2 data here as these products typically offer the maximum detail (in terms of spectral and spatial resolution) and greatest flexibility for developing our own processing. This is helpful when looking at ocean colour applications in complex waters.

Let's take a look at this data to understand more about what it can tell us about water quality.

<div class="alert alert-info" role="alert">

## 2. <a id='section2'></a>Setting up our experiment
[Back to top](#TOC_TOP)

</div>

To start with, we are going to set some parameters for running our notebook in order to control image management and plotting. You do not need to alter anything here, but if you find that you are having memory problems on your local system you can increase the grid_factor parameter and/or reduce the size of your subset. You can also set these parameters to false if you want to try running with a full image!

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../../../README.md" target="_blank">README</a>**.

In [1]:
import os                                  # a library that allows us access to basic operating system commands
import sys                                 # a library that gives us access to various command line tools to support pathing
import shutil                              # a library that allows us access to basic operating system commands like copy
import cartopy                             # a library that supports mapping and projection
import fnmatch                             # a library to allow us to discover files in our filesystem MAY NOT NEED THIS WITH XARRAY OPEN_MF
from matplotlib import gridspec            # a library that gives us better control over plot spacing
import matplotlib.pyplot as plt            # a library the provides plotting capability
import numpy as np                         # a library that lets us work with arrays; we import this with a new name "np"
import xarray as xr                        # a library that helps us work efficiently with multi-dimensional arrays
from sklearn.naive_bayes import GaussianNB # the library for constructing naive bayesian classifiers 
import zipfile                             # a library that allows us to unzip zip-files.
import eumdac                              # a tool that helps us download via the eumetsat/data-store
from pathlib import Path                   # a library that helps construct system path objects
import getpass                             # a library to help us enter passwords
import warnings                            # a library that supports managing warning messages

warnings.filterwarnings('ignore')
plt.rc('font', size=10)
sys.path.append(os.path.dirname(os.getcwd()))
import common_functions as osf             # an accompanying library of bespoke functions to support this notebook

In [ ]:
download_dir = os.path.join(os.getcwd(), "Support_data")
download_data = False

In [ ]:
# image reduction settings: resample the image every grid_factor points
run_reduce_image = True
grid_factor = 5

# subset image: cut a relevant section out of an image. subset_extents [lon1,lon2,lat1,lat2] describes the section.
run_subset_image = True
subset_extents = [-2.0, 12.0, 51.75, 59.25]

# get our land mask from NaturalEarth
land_poly = cartopy.feature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='k', facecolor=cartopy.feature.COLORS['land'])

Below we define three spatial boxes, over which we will compare OLCI spectra. We also set the maximum band value (in nm) for the spectral data you will use to train the model in this workflow.

In [ ]:
# spectral_box_extents: Selects are area (or point) to extract spectra from
spectral_box_extents = [[10.0, 10.5, 57.75, 58.25], [3.5, 4.0, 53.25, 53.75], [7.75,8.25,54.25,54.75]]
spectral_plot_cols = ['#00FFFF','#FF6103', '#8A2BE2']

# maximum band to match OLCI with training data set
band_max = 760

<div class="alert alert-info" role="alert">

## 3. <a id='section3'></a>Downloading from the Data Store via API >>>> CHANGE TO WEKEO?
[Back to top](#TOC_TOP)

</div>

<div class="alert alert-block alert-success">

### Accessing the EUMETSAT Data Store

To access Copernicus marine data from the <a href="https://data.eumetsat.int " target="_blank">EUMETSAT Data Store</a>, we will use the EUMETSAT Data Access Client (`eumdac`) python package. If you are working with the recommended Anaconda Python distribution and used the environment file included in this repository (environment.yml) to build this python environment (as detailed in the README), you will already have installed this. If not, you can install eumdac using;

`conda install -c eumetsat eumdac`

You can also find the source code on the <a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac " target="_blank">EUMETSAT GitLab</a>. Please visit the EUMETSAT user portal for more information on the <a href="https://user.eumetsat.int/data-access/data-store " target="_blank">EUMETSAT Data Store</a> and <a href="https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide " target="_blank">eumdac</a>.

To download data from the EUMETSDAT Data Store via API, you need to provide credentials. To obtain these you should first register at for an <a href="https://eoportal.eumetsat.int/" target="_blank">EUMETSAT Earth Observation Portal account</a>. Once you have an account, you can retrieve your `<your_consumer_key>` and `<your_consumer_secret>` from the <a href="https://api.eumetsat.int/api-key/ " target="_blank">"EUMETSAT Data Store API"</a> page (*Note: you must click the "Show hidden fields" button at the bottom of the page to see the relevant fields*). If you do not already have a local credentials file, you will be prompted to enter your credentials when you run the cell below. This will create the required local credentials file, so that you only need to run this once.

*Note: your key and secret are permanent, so you should take care to never share them*

</div>

In [ ]:
if download_data:
    
    # load credentials
    eumdac_credentials_file = Path(Path.home() / '.eumdac' / 'credentials')
    
    if os.path.exists(eumdac_credentials_file):
        consumer_key, consumer_secret = Path(eumdac_credentials_file).read_text().split(',')
    else:
        # creating authentication file
        consumer_key = input('Enter your consumer key: ')
        consumer_secret = getpass.getpass('Enter your consumer secret: ')
        try:
            os.makedirs(os.path.dirname(eumdac_credentials_file), exist_ok=True)
            with open(eumdac_credentials_file, "w") as f:
                f.write(f'{consumer_key},{consumer_secret}')
        except:
            pass
            
    token = eumdac.AccessToken((consumer_key, consumer_secret))
    print(f"This token '{token}' expires {token.expiration}")

Now we have a token, we can create an instance of the EUMETSAT Data Store.

In [ ]:
if download_data:
    datastore = eumdac.DataStore(token)

In [ ]:
product_dict = {}
product_dict['L1B'] = {'collectionID': 'EO:EUM:DAT:0409',
                       'productID': 'S3A_OL_1_EFR____20210416T095514_20210416T095814_20210417T142009_0179_070_350_1980_MAR_O_NT_002.SEN3'}
product_dict['L2'] = {'collectionID': 'EO:EUM:DAT:0556',
                      'productID': 'S3A_OL_2_WFR____20210416T095514_20210416T095814_20210817T023423_0179_070_350______MAR_R_NT_003.SEN3'}

In [ ]:
if download_data:
    for item in product_dict:
        selected_product = datastore.get_product(product_id=product_dict[item]['productID'], collection_id=product_dict[item]['collectionID'])
        
        with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
            print(f'Downloading {fsrc.name}')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {fsrc.name} finished.')
    
        with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.startswith(str(selected_product)):
                    zip_ref.extract(file, download_dir)
            print(f'Unzipping of product {selected_product} finished.')
        os.remove(fdst.name)

<div class="alert alert-info" role="alert">

## 4. <a id='section4'></a>Loading and checking our data
[Back to top](#TOC_TOP)

</div>

You will begin by setting a few parameters, such as `DATA_PATHS` that point to where you can find the example OLCI data. 

In [ ]:
DATA_PATHS = [os.path.join(download_dir, product_dict['L1B']['productID']),
              os.path.join(download_dir, product_dict['L2']['productID'])]

To get an overall picture of how this data looks for this place and time, you can create what is known as an `RGB` image. The 'RGB' stands for Red, Green, Blue, because, to create these type of images, you select data from red, green, and blue wavebands. In the next section of code, you read in the relevant coordinate data (latitude and longitude) and the data from the radiance (level-1) and reflectance (level-2) bands. Lastly you create combinations of red, green, and blue bands, here using a recipe called ['tristimulus'](https://www.britannica.com/science/tristimulus-system).

In [ ]:
# read channels

# >>>> replace with eumartools routines!! >> move any other functions to the top of this notebook

lon, lat, red_L1, green_L1, blue_L1, rlon, rlat = osf.get_OLCI_RGB(DATA_PATHS[0],\
                                         run_reduce_image=True,\
                                         run_subset_image=run_subset_image,\
                                         subset_extents=subset_extents,\
                                         return_orig_coords=True)

lon, lat, red_L2, green_L2, blue_L2, rlon, rlat = osf.get_OLCI_RGB(DATA_PATHS[1],\
                                         run_reduce_image=True,\
                                         run_subset_image=run_subset_image,\
                                         subset_extents=subset_extents,\
                                         return_orig_coords=True)

In this next section you use a few functions to make it easier to plot the data in the interactive environment of the Jupyter Notebook. If you wish to understand this process in more detail, you can see the functions in the accompanying file - [ocean_support_functions_4F.py](./ocean_support_functions_4F.py), which you will find in the same folder as this notebook.

In [ ]:
# >>>> replace with eumartools routines!! >> move any other functions to the top of this notebook

image_array_L1, colorTuple_L1, lon, lat = osf.process_image(lon, lat, red_L1, green_L1, blue_L1,
                                          run_truncate_image=True,
                                          run_histogram_image=True,
                                          unhitch = True)

image_array_L2, colorTuple_L2, lon, lat = osf.process_image(lon, lat, red_L2, green_L2, blue_L2,
                                          run_truncate_image=True,
                                          run_histogram_image=True)

...and finally you plot both images, using functions from the cartopy library.

In [ ]:

# improve figure!!!!

# intitialise a figure
fig1 = plt.figure(figsize=(12, 5), dpi=300)
gs = gridspec.GridSpec(1, 2)

# make an axis and plot
m = plt.subplot(gs[0,0], projection=cartopy.crs.Mercator())
p1 = osf.plot_OLCI_scene(m, lon, lat, lon, run_subset_image=True, RGB_plot=True, \
                    subset_extents=subset_extents, colorTuple=colorTuple_L1, fsz=10)
# embellish with spectral boxes
osf.add_boxes(m, spectral_plot_cols, spectral_box_extents)

# make an axis
m = plt.subplot(gs[0,1], projection=cartopy.crs.Mercator())
p1 = osf.plot_OLCI_scene(m, lon, lat, lon, run_subset_image=True, RGB_plot=True, \
                    subset_extents=subset_extents, colorTuple=colorTuple_L2, fsz=10)
# embellish with spectral boxes
osf.add_boxes(m, spectral_plot_cols, spectral_box_extents)

# add caption
fig1.text(0.5, 0.0, r'Figure 1: RGB colour composites of level-1 data (left) and level-2 data (right) from'+'\n'+ 
          'OLCI on the 16th April 2021. Three boxes highlight different regions of interest.', ha='center')
plt.show()

You can use figure 1 to compare the level-1 and level-2 data, and to start to understand how different water types manifest in the ocean colour signal. 

The first thing you can note is that the figure on the left, is much bluer than the one on the right. This is predominantly the effect of the atmosphere, which is present in the level-1 data (left) but not in the level-2 (right). The level-2 RGB has also been enhanced by the choice of method you have used, so you are not seeing a true colour as such, but an enhanced RGB so that can be used to see some of the differences in the water colour more clearly. 

To focus on some of the differences, we have overlaid our 3 boxes to investigate a few regions of interest. The cyan box contains waters that show up as dark blue, the orange box a lighter blue-green colour, and the purple box more green-brown in colour. You can infer from this that the patch of ocean in the cyan box has probably got relatively low concentrations of sediments or phytoplankton, whilst the other two boxes are likely more mixed, with potentially higher concentrations of phytoplankton and/or sediments. Just looking at the images like this it is hard to say exactly what is going on, particularly in the latter two boxes. As mentioned before, a benefit of the satellite ocean colour data is that we don't have to look at it as an integrated single colour as our eyes see, we can look more at the detail. Next, let's look at the colour spectra associated with these regions of interest...

<div class="alert alert-info" role="alert">

## 5. <a id='section5'></a>Interpreting OLCI spectra
[Back to top](#TOC_TOP)

</div>

OLCI has 21 spectral bands - these are the wavelengths within the electromagnetic spectrum where the instrument senses the reflected light leaving the Earths surfaces. Some of these are useful for determining atmospheric properties (for atmospheric correction, and for composition data (see the atmosphere week for this!)), but most give us information on the ocean (and land, see the land week!) surface. In this section, we'll extract the ocean colour spectra from the OLCI data, associated with the three regions of interest we define above.

First we get the metadata information we need to extract the data at the different bands.

In [ ]:
# >>>>>> get wavelengths from xml file:: >> should improve this with proper XML parsing, not string read!

bands = [] ; wavelengths = []; bandwidths = []    
xml_file = os.path.join(DATA_PATHS[1],'xfdumanifest.xml')
with open(xml_file, 'r') as input_file:
    for line in input_file:
        if "<sentinel3:band name=" in line:
            bands.append(line.replace('<sentinel3:band name="',"").replace('">','').replace(' ','').replace('\n',''))
        if "<sentinel3:centralWavelength>" in line:
            wavelengths.append(float(line.replace('<sentinel3:centralWavelength>','').replace('</sentinel3:centralWavelength>','')))
        if "<sentinel3:bandwidth>" in line:
            bandwidths.append(float(line.replace('<sentinel3:bandwidth>','').replace('</sentinel3:bandwidth>',''))/2)

Now we will read in the reflectances for our box for all the bands. Here we read in all the spectra over each of our boxes, for every wavelength and make an average spectra for each box.

In [ ]:
# get all the reflectance data
ds = xr.open_mfdataset(os.path.join(DATA_PATHS[1], "Oa*.nc"))

# get the reflectances for our boxes or points
all_reflectances = []; all_reflectance_errors = []; all_variability = []; all_uppers = []; all_lowers = []

for spectral_box_extent in spectral_box_extents:
    print('Processing box...')
    I1, I2, J1, J2 = osf.get_coords(rlon, spectral_box_extent[0], spectral_box_extent[1],
                                    rlat, spectral_box_extent[2], spectral_box_extent[3], False)

    reflectances = []; reflectance_errors = []; variability = []
    for item in ds:
        if "_err" in item:
            reflectance_errors.append(np.nanmean(ds[item][I1:I2,J1:J2]))
        else:
            reflectances.append(np.nanmean(ds[item][I1:I2,J1:J2]))
            variability.append(np.nanstd(ds[item][I1:I2,J1:J2]))

    all_uppers.append([x + y for x, y in zip(reflectances, variability)])
    all_lowers.append([x - y for x, y in zip(reflectances, variability)])
    all_reflectances.append(reflectances)
    all_reflectance_errors.append(reflectance_errors)
    all_variability.append(variability)

Now we plot these spectra.

In [ ]:
# initialise a figure
fig1 = plt.figure(figsize=(12, 5), dpi=300)

# gather our spectra and plot
for reflectances, reflectance_errors, uppers, lowers, spectral_plot_col in \
  zip(all_reflectances, all_reflectance_errors, all_uppers, all_lowers, spectral_plot_cols):
    plt.errorbar(wavelengths, reflectances, xerr=bandwidths, yerr=abs(np.array(reflectance_errors)),
      color=spectral_plot_col, linewidth=1.0)
    plt.plot(wavelengths, uppers, color=spectral_plot_col, linestyle='--',linewidth=1.0)
    plt.plot(wavelengths, lowers, color=spectral_plot_col, linestyle='--',linewidth=1.0)
    plt.fill_between(wavelengths,uppers,lowers,color=spectral_plot_col,alpha=0.25)
    
plt.xlabel('Wavelength [nm]')
plt.ylabel('Water leaving reflectance [sr$^{-1}$]')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('OLCI L2 reflectances')

# add caption
fig1.text(0.5, -0.175, r'Figure 2: OLCI level-2 water leaving reflectance spectra. Solid trace shows the mean'+'\n'+
          r'spectrum, with +/- 1 standard deviation covered by the shaded area. Horizontal and vertical'+ '\n' +
          r'bars show the respective band widths and reflectance errors for each wavelength.' +'\n' +
          r'Note that water leaving reflectance ($\rho_w$) can be converted to remote sensing' +'\n' +
          r'reflectance ($R_{rs}$) by dividing by $\pi$ and correcting for BRDF.', ha='center')
plt.show()

### Challenges interpreting spectra

In figure 2 you can see the average (and range of) reflectance spectra for each of the regions of interest. We can make a few simple observations which will tell us a bit more about what's likely to be in the water at each of these locations. 

Firstly, the spectra from the orange and purple boxes are much higher in general. This tells us that there is more reflection from the water in these areas. Increased reflection typically means that we have a great proportion of components that scatter in the water - this is often sediments, but can also be from certain types of phytoplankton such as coccolithophores, or species with very small cells or complex internal structures. 

Secondly we can note that there is substantial overlap in the ranges of reflectance between the regions in the orange and purple boxes. This implies that, while there may be some differences, it might be difficult to distinguish between these regions and extract information about the water properties.

A third thing we can observe is a small feature in the 665-710 nm range. This feature is associated with chlorophyll-a absorption and fluorescence, and this indicates that there is at least some phytoplankton activity in all the regions. 

Interpreting what these spectra mean can be challenging, but is the basis of all quantitative retrievals of ocean parameters from this data. It is very difficult to fully interpret a spectra by eye, and would be very laborious to do for the high number of pixels that make up an image. Machine learning can help us to improve how we classify water types, and how we choose methods to apply to calculate properties we are interested in to estimate concentrations of different constituents like phytoplankton and sediments. In the next section, we'll look at how we can train a simple model to identify two different water types. 

<div class="alert alert-info" role="alert">

## 6. <a id='section6'></a>Classifying spectra with a Naive Bayesian Classifier
[Back to top](#TOC_TOP)

</div>

As discussed in previous weeks and notebook examples, an essential part of any machine learning workflow, is developing a training data set for the model you wish to use. The choice of training data can be a key determinant of the success of the model when it is applied. 

From an ocean colour perspective, this means that you need to have training data that represents the variety of different optical properties found in ocean waters. In many remote sensing applications, training data is gathered by visual inspection of images to identify features that we are interested in, but in the ocean this is not really possible. As shown in this workflow so far, it is not necessarily easy or reliable for us to classify ocean colour images with our own eyes, and much more information is available from spectrally resolved data that can be used to inform the retrieval of geophysical parameters. As a result of this, applying machine learning to ocean colour data often requires the use of in situ or modelled data sets that characterise ocean colour spectra in relation to ocean properties of interest such as concentrations of chlorophyll-a, sediments, or coloured dissolved organic matter, all of which are useful for ocean water quality management.

In this section you will work through a simplified example with a training data set to see if you can classify pixels within ocean colour images in to two broad types, based on whether they have high levels of sediments and dissolved organic matter (as is common in coastal regions) or not. 

### Training data 

The training data we will work with here comes from a combination of models that allows for simulation of reflectance spectra as measured by satellite instruments like OLCI. The models used in this instance include an inherent optical property model, which determines the absorption and scattering properties of the water based on a choice of parameters; and a radiative transfer model, which takes these absorption and scattering properties and calculates the reflectance (ocean colour) signal. For more information on the models used to generate this data set, see Evers-King et al., 2014. 

For this example we have chosen this training data set to see if we can classify an image in to two broad water types linked to what we observed above. The first water type, which we shall refer to as case-1, does not have lots of sediments and coloured dissolved organic matter in. These are types of water that might exist in the open ocean, or in coastal areas without strong riverine inputs. The second type, which we will call case-2, does have a lot of these non-phytoplankton constituents. 
Please note that this data set is for demonstration purposes only, and is not appropriate for wider scientific use in the form provided here. 

For any machine learning application using ocean colour data you could create a training data set using your own models, depending on what constituents you wanted to investigate, and there are many tools and literature examples that can support this. You could also gather a data set from coincident in situ data - this can be more representative of reality, because all models come with assumptions, however it can be hard to gather enough data points to be representative of the real variability of the parameters of interest in the ocean. See the next section for an example which uses a very extensive in situ and model data set in its training approach. 

First let's open the training data set for this example workflow. The cells below set our data files and then open and read in our training data.

In [ ]:
training_data = os.path.join(os.getcwd(), 'Support_data', 'Modelled_spectral_reflectance_data.nc')
eco = xr.open_dataset(training_data)
ECO_wavs = eco.wavelength # these are our wavelengths
ECO_Rrs = eco.RRS_MOD # these are our modelled remote sensing reflectances
ECO_LNLG = eco.LNLG # this is our flag our low non-algal, now CDOM waters
ECO_HNHG = eco.HNHG # this is our flag for high non-algal, high CDOM waters >> e.g. complex waters!
eco.close()

Now let's plot the modelled ocean colour spectra that make up the training data set...

In [ ]:
# initialise a figure
fig1 = plt.figure(figsize=(12, 5), dpi=300)

# gather our spectra and plot
for ii in range(0,np.shape(ECO_Rrs)[1]):
    if ECO_LNLG[ii] == 1:
        plt.plot(ECO_wavs, ECO_Rrs[:,ii], 'b', linewidth=1.0, alpha=0.5, zorder=1)
    else:
        plt.plot(ECO_wavs, ECO_Rrs[:,ii], 'r', linewidth=1.0, alpha=0.5, zorder=2)
        
for wv, bd in zip(wavelengths, bandwidths):
    if wv < band_max:
        plt.fill([wv-bd/2, wv-bd/2, wv+bd/2,wv+bd/2], [0,0.2,0.2,0],color='0.85', zorder=0)

# add caption
fig1.text(.5, -0.05, r'Figure 3: Modelled ocean colour spectra from the training data set.'+'\n'
          +'Spectra in red correspond to a high non-phytoplankton particle '+'\n'
          +'scattering/high CDOM absorption (HNHG) scenario. Blue spectra are from the'+'\n'
          +'corresponding low scenario (LNLG). The grey bars show the OLCI bands.', ha='center')

plt.show()

How do we interpret these spectra? Well, based on what we learned before we can see that we have some spectra with generally higher magnitude (most of the red spectra) and others that are generally lower (in blue). This corresponds with what we know about highly scattering waters and waters with lower scattering and/or high absorption respectively. And indeed, this corresponds with the parameters that were used to generate the modelled spectra where blue corresponds to the case-1 scenario and red, the case-2 scenario. The variability that you see in the two cases, comes from model scenarios that were run to include changes in the phytoplankton present in the water (represented by chlorophyll-a concentrations and the effective size of the cells in the assemblage). Whilst not a fully comprehensive representation of all possible combinations of chlorophyll-a concentration, phytoplankton size, scattering from sediments, and absorption from dissolved organic matter, the training data here should be representative enough to roughly classify images in to the two types described, based on the reflectance spectra of each pixel. 

### Training the model

To train the model, you will first arrange the training data set so it is in the same format as the OLCI bands that we ultimately want to apply the model to.

In [ ]:
data_array = []
for wav in wavelengths:
    if wav < band_max:
        ii = np.argmin(abs(ECO_wavs.values-wav))
        data_array.append(ECO_Rrs[ii,:])
data_array = np.array(data_array).transpose()

In the next section of code, you will split the data set to keep some data for validation. This is common in machine learning as it allows you to more independently verify the performance of your model. 

In [ ]:
# split into testing and training arrays: put one third in the test data and keeping 2/3 for training
test_indices = np.zeros(ECO_HNHG.size, dtype=bool)
test_indices[::3] = True

...then you provide the training data to the `naive Bayes classification` function provided in the `scikit-learn` Python library to generate the model you want to use to classify the images. We feed the model the remote sensing reflectance values and our complex waters flag.

In [ ]:
# Train model
GNB = GaussianNB() 
class_model = GNB.fit(data_array[test_indices==False,:], ECO_HNHG[test_indices==False])

Now lets check how well our test data fits our training data.

In [ ]:
# Rough idea of how well the model output fits the training data
score = class_model.score(data_array[test_indices==True,:], ECO_HNHG[test_indices==True])
print("Naive Bayes Classification Fit: {:.02}".format(score))

This is not a reasonable fit to our model, given the complexity of the input data. It is enough for us to continue.

### Applying the model to an image

Now that you have trained the model for classifying based on the spectra from the training data set, you can use the model to classify other spectra, such as those that exist at each pixel in an image from OLCI.

To start with, you open the image and read in a spatial subset in to an array...

In [ ]:
# Read in data and arrange for classifier input
num_channels = 12
image_array = np.zeros([num_channels,len(lon.ravel())])

if run_subset_image:
    i1, i2, j1, j2 = osf.subset_image(rlat, rlon, subset_extents)

for rad_channel_number in range(1, num_channels+1):
    print('Reading {}'.format(str(wavelengths[rad_channel_number-1])))
    rad_channel = 'Oa%s_%s' % (str(rad_channel_number).zfill(2),'reflectance')
    rad_file = os.path.join(DATA_PATHS[1], rad_channel + '.nc') 
    rad_fid = xr.open_dataset(rad_file)
    exec("this_channel = rad_fid.%s.data" % (rad_channel))
    rad_fid.close()
    this_channel = this_channel / np.pi
    if run_subset_image:
        this_channel = this_channel[i1:i2,j1:j2]
    if run_reduce_image:
        this_channel = osf.reduce_image(this_channel, grid_factor=grid_factor)
    image_array[rad_channel_number - 1,:] = this_channel.ravel()

image_array = np.array(image_array).transpose()

Next, you screen out any spectra we don't want to classify because they are missing data (i.e. are flagged and therefore not processed in the underlying image). Then you use the model you trained to classify the data from the image, and finally reshape it for plotting.

In [ ]:
# screen out spectra with any bad data (e.g. NaN values)
output_array = np.zeros(np.shape(image_array)[0])*np.nan
ii = np.where(np.isfinite(np.sum(image_array, axis=1)))[0]
image_array_QC = image_array[ii,:]

# classify based on our model
outclasses = class_model.predict(image_array_QC)

# re-fit to original array size for plotting
output_array[ii] = outclasses
output_array = output_array.reshape(np.shape(lon))

The next section of code enables you to create a figure to show the output of applying the trained model to the OLCI image that you looked at earlier in this workflow.

In [ ]:
# intitialise a figure
fig1 = plt.figure(figsize=(12, 5), dpi=300)
gs = gridspec.GridSpec(1, 2)

# make an axis and plot
m = plt.subplot(gs[0,0], projection=cartopy.crs.Mercator())
p1 = osf.plot_OLCI_scene(m, lon, lat, lon, run_subset_image=True, RGB_plot = True, \
                    subset_extents=subset_extents, colorTuple=colorTuple_L2, fsz=10)
# embellish with spectral boxes
osf.add_boxes(m, spectral_plot_cols, spectral_box_extents)

# make an axis and plot
m = plt.subplot(gs[0,1], projection=cartopy.crs.Mercator())
p1 = osf.plot_OLCI_scene(m, lon, lat, output_array, run_subset_image=True, \
                    subset_extents=subset_extents,  cmap='jet', fsz=10)
# embellish with spectral boxes
osf.add_boxes(m, spectral_plot_cols, spectral_box_extents)

# add caption
fig1.text(0.5, 0.0, r'Figure 3: RGB using Level-2 ocean colour data (left),'+'\n'\
          +'classified image showing pixels classified in to case-1 (blue) and case-2 (red) '+'\n'\
          'waters using the trained model (right)', ha='center')
plt.show()

The output image (Figure 3, right) shows whether each pixel in the image corresponds more to the case-1 or case-2 data groups in the training data set. The classifier clearly shows sediment dominated waters where we might expect them - i.e. near the coasts and where there are large river outflows. You can see that both the orange and purple boxes are classified as case-2 here. However the classification also highlights some areas that are more unexpected, such as in the middle of the North sea. In this instance it is likely that these areas include regions where coccolithophores are blooming. Coccolithophores are a type of phytoplankton that forms calcium carbonate "lith" structures to protect their bodies. Whilst they do contain chlorophyll-a, the liths scatter a lot of light, making waters with coccolithophores in highly reflective more akin to sediment dominated waters. This complexity, along with the fact that we cannot distinguish any differences between the case-2 pixels such as those in the orange and purple boxes, shows some of the limitations of our simple example.


### Is this useful? 

This sort of case 1/case 2 water typing, is a basic form of classification that has a long history in the science of ocean colour. You can read more about how this classification arose, and whether it is still useful in an article by <a href="http://misclab.umeoce.maine.edu/documents/ocean_17.2_mobley_lo.pdf" target="_blank">Mobley et al. 2003</a>.

A further issue we have with this example, is the assumption of independence between features that is inherent in the use of the Naive Bayes classifier. With the modelled data we have used, we have two distinct classes we have created with the model conditions, however the reality in the ocean is much more complicated. As Mobley et al. 2003, discuss, the case 1, case 2 definition doesn't really hold, as there are many 'in-between' water types. Further, ocean colour spectra that are similar, to such a degree that they may be indistinguishable, can occur with different combinations of constituents. This inherent ambiguity makes it difficult to apply simple classification and obtain accurate results (for more information about this challenge, see <a href="http://omtab.obs-vlfr.fr/fichiers_PDF/Defoin_and_chami_JGR_07.pdf" target="_blank">Defoin-Platel and Chami, 2007</a>.

In short, simple classification can be useful conceptually and could be beneficial for applications that use broad metrics. However, for more accurate derivation of quantitative estimates of ocean parameters from ocean colour, we can do better, and indeed there are approaches that have been designed to do just that in the <a href="../Neural_network_complex_water_parameters_C2RCC/Using_ML_to_retrieve_water_quality_indicators_in_coastal_waters.ipynb" target="_blank">C2RCC neural network example.


<div class="alert alert-info" role="alert">

## 7. <a id='section7'></a>Conclusions
[Back to top](#TOC_TOP)

</div>

For decision making about activities in our coastal oceans, data is often needed on water quality. Satellite ocean colour data can provide this information, but deriving the parameters needed from the signal can be complex because of the way that the optical properties of different constituents overlap. Machine learning approaches can be developed using in situ and modelled data sets if they are representative of the wide variety of optical properties that occur in the oceans. Ultimately we can then retrieve estimates of parameters such as chlorophyll-a and sediment concentrations which allow us to monitor ocean impacts, plan marine activities, and meet international policy objectives for sustainable seas. 

For further examples of how Copernicus satellite data is used in water quality applications see the relevant pages of the <a href="https://www.copernicus.eu/en/website-search?search_api_fulltext=water+quality" target="_blank">Copernicus website</a>.

<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>